This notebook is inspired by Jerry Ling's documentation:
https://moelf.github.io/WVZAnalysis.jl/dev/

In [1]:
using WVZAnalysis, UnROOT, FHist, Printf, Arrow, Serialization, UnROOT

# Pkg.activate(pathof(WVZAnalysis) |> dirname |> dirname)

    CondaPkg Found dependencies: /home/grabanal/WVZAnalysis.jl/CondaPkg.toml
    CondaPkg Found dependencies: /home/grabanal/.julia/packages/PythonCall/eU0yr/CondaPkg.toml
    CondaPkg Dependencies already up to date


These are all the constant directories presently stored, and they can be changed by the user:

In [2]:
@show WVZAnalysis.MINITREE_DIR[] # this is where the code will look for input root files
@show WVZAnalysis.ANALYSIS_DIR[] # this is where the code will store results
@show WVZAnalysis.ONNX_MODEL_PATH[] # this is where the code will load a NN model
@show WVZAnalysis.BDT_MODEL_PATH[]  # this is where the code will load a BDT model

WVZAnalysis.MINITREE_DIR[] = "/data/jiling/WVZ/v2.3"
WVZAnalysis.ANALYSIS_DIR[] = "/data/jiling/WVZ/v2.3_hists"
WVZAnalysis.ONNX_MODEL_PATH[] = "/data/grabanal/NN/NN_08_23.onnx"
WVZAnalysis.BDT_MODEL_PATH[] = "/data/jiling/WVZ/v2.3-beta2_arrow/xgb_2022-10-17_jerry.model"


"/data/jiling/WVZ/v2.3-beta2_arrow/xgb_2022-10-17_jerry.model"

The full analysis can create either histograms or dictionaries. 
- The "histogram mode" makes a triplet histograms of NN/BDT score, one for each signal region (SF-inZ, SF-noZ, DF). This  can later be stored as `.root` files and used by TRExFitter.
- The "dictionary mode" makes a dictionary of physical observables (pt, eta, phi, etc.) as well as weight *for all entries* that pass the analysis cuts, entry by entry. Those dictionaries can be later stored as versatile `.arrow` files and can be imported as dataframes for further studies in Julia, Python, etc.

You will set the "histogram mode" by `NN_hist = true` and `arrow_making = false`. 

You will set the "dictionary mode" by `NN_hist = false` and `arrow_making = true`. 

# 1. Running the analysis only on nominal "Signal"

To get the yields only for VVZ, we can use the `"Signal"` tag that is defined in `config/file_list.json`. All the tags that can be shown in `WVZAnalysis.ALL_TAGS`

In [3]:
WVZAnalysis.ALL_TAGS

("Signal", "ZZ", "Zjets", "Zgamma", "ttbar", "WZ", "tZ", "ttZ", "tWZ", "VBS", "VH", "Others")

In order to run the analysis on `"Signal"`, we have to: 
- create one task for each `"Signal"` sub-file using the function `prep_tasks`
- use the function `main_looper` (the analysis) on those tasks, making a vector of results
- add/merge the results

To create a task, the function `prep_tasks` needs to know whether the analysis will create histograms or dictionaries.

#### a) Using the histogram mode

In [4]:
signal_tasks = prep_tasks("Signal"; arrow_making=false, NN_hist=true, shape_variation="NOMINAL"); 
vector = map(main_looper, signal_tasks)
res = reduce(mergewith(+), deepcopy(vector))

N     = nbins(res[:DF__NN__NOMINAL])
hists = rebin.([res[:SFinZ__NN__NOMINAL], res[:SFnoZ__NN__NOMINAL], res[:DF__NN__NOMINAL]], N);

@printf("Signal yield: \n")
@printf("- in SF-inZ:  %0.2f ± %0.2f\n", integral(hists[1]), only(binerrors(hists[1])))
@printf("- in SF-noZ:  %0.2f ± %0.2f\n", integral(hists[2]), only(binerrors(hists[2])))
@printf("- in DF:      %0.2f ± %0.2f\n", integral(hists[3]), only(binerrors(hists[3])))
@printf("Total:        %0.2f ± %0.2f\n", sum(map(integral,hists)), sqrt(sum(map(only∘binerrors, hists).^2)) )

Signal yield: 
- in SF-inZ:  11.73 ± 0.07
- in SF-noZ:  9.31 ± 0.10
- in DF:      10.73 ± 0.14
Total:        31.77 ± 0.19


You can save the result as `.jlserialize` or `.root`

In [ ]:
# Name for the root file
name = "test_hist" 
# Target folder
p = WVZAnalysis.ANALYSIS_DIR[] 

#################################
### Saving it as .jlserialize ###
#################################
# using Serialization
# serialize(joinpath(p,"$(name).jlserialize"), res)

##########################
### Saving it as .root ###
##########################
# using PythonCall

# function make_TH1D(h)
#     np = pyimport("numpy")
#     pyhist = pyimport("hist")

#     hout = pyhist.Hist(pyhist.axis.Regular(100, 0, 1), storage=pyhist.storage.Weight())
#     bc = bincounts(h)
#     va = h.sumw2
#     hout[pybuiltins.Ellipsis] = np.stack([bc, va], axis=-1)
#     return hout
# end

# up = pyimport("uproot")
# pywith(up.recreate(joinpath(p, "$(name).root"))) do file
#     for (k,v) in res
#         file[string(k)] = make_TH1D(v)
#     end
# end

The analysis package actually has functions that can do these things for all processes and all systematics in just a few of lines. It just takes several more lines in this example because I am focusing on the details involved in processing a single process

#### b) Using the dictionary mode

In [ ]:
signal_tasks = prep_tasks("Signal"; arrow_making=true, NN_hist=false, shape_variation="NOMINAL"); 
vector = map(main_looper, signal_tasks)
res = reduce(mergewith(append!), deepcopy(vector))

@printf("Signal yield: \n")
@printf("- in SF-inZ:  %0.2f ± %0.2f\n", sum(res[:wgt][res[:SR] .== 0]), sqrt(sum(res[:wgt][res[:SR] .== 0].^2)))
@printf("- in SF-noZ:  %0.2f ± %0.2f\n", sum(res[:wgt][res[:SR] .== 1]), sqrt(sum(res[:wgt][res[:SR] .== 1].^2)))
@printf("- in DF:      %0.2f ± %0.2f\n", sum(res[:wgt][res[:SR] .== 2]), sqrt(sum(res[:wgt][res[:SR] .== 2].^2)))
@printf("Total:        %0.2f ± %0.2f\n", sum(res[:wgt]), sqrt(sum(res[:wgt].^2)))

You can save the result as `.jlserialize` or `.arrow`

In [1]:
# Name for the root file
name = "test_dict" 
# Target folder
p = WVZAnalysis.ANALYSIS_DIR[] 

#################################
### Saving it as .jlserialize ###
#################################
# using Serialization
# serialize(joinpath(p,"$(name).jlserialize"), res)

###########################
### Saving it as .arrow ###
###########################
# using Arrow
# Arrow.write(joinpath(p,"$(name).arrow"), res)

LoadError: UndefVarError: WVZAnalysis not defined

The analysis package actually has functions that can do these things for all processes and all systematics in just a few of lines. It just takes several more lines in this example because I am focusing on the details involved in processing a single process

# 2. Significance table of all processes in two lines

All posible tags can be found in `config/file_list.json`, it is there where we define the paths for the root files of "Signal", "ZZ", "WZ", etc.

The following line allows you to analyze at once all the physics processes (signal and backgrounds) in each signal region, as well as their significance in quadrature and print it out as a nice yield table. Using a single thread, this line takes just about 30 minutes with `recreate=true`. The way this works is that internally, the function `significance_table` calls upon the `prep_tasks` and `main_looper` as described in the section above, except it does it for all processes.

This will also store the results of the analysis as `.jlserialize` files in the directory `WVZAnalysis.ANALYSIS_DIR[]` so that for the next runs of the code, we can just read those files using `recreate=false` and this will take only ~15 seconds.

In [7]:
WVZAnalysis.ANALYSIS_DIR[] = "/data/grabanal/WVZ/v2.3_hists"
WVZAnalysis.ANALYSIS_DIR[]

"/data/grabanal/WVZ/v2.3_hists"

The following cell will not work if it's the first time you run the analysis 

In [9]:
# @time begin 
#     M = significance_table(; recreate=false);
#     print_sigtable(M)
# end

┌───────────────┬────────────────┬───────────────┬──────────────┐
│               │     SF-inZ     │    SF-noZ     │      DF      │
├───────────────┼────────────────┼───────────────┼──────────────┤
│    Signal     │  11.73 ± 0.07  │  9.31 ± 0.1   │ 10.73 ± 0.14 │
│      ZZ       │ 1775.6 ± 5.89  │ 469.06 ± 2.44 │ 19.78 ± 0.45 │
│     Zjets     │  -0.02 ± 0.13  │  2.6 ± 2.22   │ 6.47 ± 5.51  │
│    Zgamma     │   0.0 ± 0.0    │   0.0 ± 0.0   │  0.3 ± 0.29  │
│     ttbar     │   0.0 ± 0.0    │  0.63 ± 0.18  │  0.28 ± 0.1  │
│      WZ       │   0.36 ± 0.1   │  1.79 ± 0.23  │ 2.24 ± 0.29  │
│      tZ       │  0.01 ± 0.01   │  0.07 ± 0.03  │ 0.06 ± 0.02  │
│      ttZ      │  1.24 ± 0.08   │  4.71 ± 0.16  │ 5.74 ± 0.18  │
│      tWZ      │  0.57 ± 0.11   │  2.16 ± 0.23  │  2.5 ± 0.24  │
│      VBS      │  11.71 ± 0.09  │  6.4 ± 0.08   │ 0.18 ± 0.01  │
│      VH       │  1.29 ± 0.71   │  5.76 ± 1.4   │ 5.77 ± 1.29  │
│    Others     │  0.06 ± 0.01   │  0.4 ± 0.13   │ 0.56 ± 0.08  │
├─────────

The following cell will recreate all the `.jlserialize` files for all the nominal processes *and* make the significance table. It can take a while (at least 25 minutes)

In [10]:
# @time begin 
#     M = significance_table(; recreate=true);
#     print_sigtable(M)
# end

# 3. Analyzing a single DSID / MC Campaign

For this example, I will analyze a ZZ file with DSID 364250 for the mc16a campaign.

In [54]:
dir_path = "/data/rjacobse/WVZ/v2.3/user.jiling.WVZ_v2.3sf.364250.e5894_s3126_r9364_p4434_ANALYSIS.root/";

Looking at the filename ending, it looks like a `.root` file. However, it is actually a folder that contains several `.root` files because this is how the minitrees are produced.

In [55]:
readdir(dir_path)

4-element Vector{String}:
 "sumsumWeight.txt"
 "user.jiling.29896114._000001.ANALYSIS.root"
 "user.jiling.29896114._000002.ANALYSIS.root"
 "user.jiling.29896114._000003.ANALYSIS.root"

Point in fact, there are *three* `.root` files inside this folder (there is also a `.txt` file which can be ignored for now)

Each one of them has a quantity `sumWeight`. The entries in all these three files belong to the same DSID and MC campaign so this is why we need to sum all the `sumWeight` of each `.root` file to get the "real" sum of weights. It is this quantity, that we call `sumsumWeight` that the MC weights will have to be divided by. This is how it looks like for these files:

In [58]:
sumsumWeight = 0.0
for f in readdir(dir_path; join=true)
    if endswith(f, ".root")
        r = ROOTFile(f)
        sumsumWeight += r["sumWeight"][:fN][3]
    end
end

sumsumWeight

7.518811876953125e6

In [59]:
# Another way
subfiles = readdir(dir_path; join=true)
subfiles = filter(endswith(".root"), subfiles)
sumsumWeight = mapreduce(x -> ROOTFile(x)["sumWeight"][:fN][3], +, subfiles)

7.518811876953125e6

Now I will make an `AnalysisTask` object. 
This is the atomic unit of runnable task, it runs on a single `.root` file and it needs the `sumsumWeights` (and other optional arguments) as input to fully apply the analysis to this file. 
For this example, the output of the analysis will be a dictionary of physical observables.
Since the `AnalysisTask` object can run on a single `.root` file at a time, I can do this:

In [60]:
root_path = joinpath(dir_path, "user.jiling.29896114._000001.ANALYSIS.root");

my_task = AnalysisTask(path            = root_path, 
                       sumWeight       = sumsumWeight,
                       arrow_making    = false, # We are choosing to make histograms this time but feel free to change
                       NN_hist         = true,  # We are choosing to make histograms this time but feel free to change
                       shape_variation = "NOMINAL");

res = main_looper(my_task)

Dict{Symbol, Hist1D} with 3 entries:
  :SFinZ__NN__NOMINAL => Hist1D{Float64}, edges=0.0:0.01:1.0, integral=87.90420…
  :DF__NN__NOMINAL    => Hist1D{Float64}, edges=0.0:0.01:1.0, integral=0.946533…
  :SFnoZ__NN__NOMINAL => Hist1D{Float64}, edges=0.0:0.01:1.0, integral=24.30252…

Now, `main_looper` is running the full analysis on this `AnalysisTask`.

- If `NN_hist = true` and `arrow_making = false`, result will be a triplet of histograms, one for each signal region (SF-inZ, SF-noZ, DF)
- If `NN_hist = false` and `arrow_making = true`, result will be a dictionary, where the "keys" are different magnitudes that we chose, and the "values" are the actual values event by event:

Getting the yield (integral):

In [61]:
#########################
### In histogram mode ###
#########################
N     = nbins(res[:DF__NN__NOMINAL])
hists = rebin.([res[:SFinZ__NN__NOMINAL], res[:SFnoZ__NN__NOMINAL], res[:DF__NN__NOMINAL]], N);
@printf("The yield of this subfile: \n")
@printf("- in SF-inZ:  %0.2f ± %0.2f\n", integral(hists[1]), only(binerrors(hists[1])))
@printf("- in SF-noZ:  %0.2f ± %0.2f\n", integral(hists[2]), only(binerrors(hists[2])))
@printf("- in DF:      %0.2f ± %0.2f\n", integral(hists[3]), only(binerrors(hists[3])))
@printf("Total:        %0.2f ± %0.2f\n", sum(map(integral,hists)), sqrt(sum(map(only∘binerrors, hists).^2)) )

##########################
### In dictionary mode ###
##########################
# @printf("Signal yield: \n")
# @printf("- in SF-inZ:  %0.2f ± %0.2f\n", sum(res[:wgt][res[:SR] .== 0]), sqrt(sum(res[:wgt][res[:SR] .== 0].^2)))
# @printf("- in SF-noZ:  %0.2f ± %0.2f\n", sum(res[:wgt][res[:SR] .== 1]), sqrt(sum(res[:wgt][res[:SR] .== 1].^2)))
# @printf("- in DF:      %0.2f ± %0.2f\n", sum(res[:wgt][res[:SR] .== 2]), sqrt(sum(res[:wgt][res[:SR] .== 2].^2)))
# @printf("Total:        %0.2f ± %0.2f\n", sum(res[:wgt]), sqrt(sum(res[:wgt].^2)))

The yield of this subfile: 
- in SF-inZ:  87.90 ± 1.18
- in SF-noZ:  24.30 ± 0.48
- in DF:      0.95 ± 0.09
Total:        113.15 ± 1.28


This processes one single sub-file.
However, if you recall there are actually 3 sub-files that need to be processed. 
I can just do a for-loop to process all 3. It is a large "ZZ" file so it could take at least around 2~3 minutes

In [62]:
vector = [] # will be a vector

# Getting sumsumWeight
sumsumWeight = 0.0
for f in readdir(dir_path; join=true)
    if endswith(f, ".root")
        r = ROOTFile(f)
        sumsumWeight += r["sumWeight"][:fN][3]
    end
end
print("sumsumWeight is ", sumsumWeight, "\n")

# Processing a task for each .root file
for f in readdir(dir_path; join=true)
    if endswith(f, ".root")
        r = ROOTFile(f)
        print("--- Analyzing ", f, " ... \n")
        myAnalysisTask = AnalysisTask(path = f, 
                                      sumWeight = sumsumWeight, # using sumsumWeight
                                      arrow_making=false, 
                                      NN_hist=true,
                                      shape_variation = "NOMINAL")
        print("--- Pushing into the vector \n")
        push!(vector, main_looper(myAnalysisTask))
    end
end

sumsumWeight is 7.518811876953125e6
--- Analyzing /data/rjacobse/WVZ/v2.3/user.jiling.WVZ_v2.3sf.364250.e5894_s3126_r9364_p4434_ANALYSIS.root/user.jiling.29896114._000001.ANALYSIS.root ... 
--- Pushing into the vector 
--- Analyzing /data/rjacobse/WVZ/v2.3/user.jiling.WVZ_v2.3sf.364250.e5894_s3126_r9364_p4434_ANALYSIS.root/user.jiling.29896114._000002.ANALYSIS.root ... 
--- Pushing into the vector 
--- Analyzing /data/rjacobse/WVZ/v2.3/user.jiling.WVZ_v2.3sf.364250.e5894_s3126_r9364_p4434_ANALYSIS.root/user.jiling.29896114._000003.ANALYSIS.root ... 
--- Pushing into the vector 


I can now `merge` all the 3 results with:

In [14]:
#########################
### In histogram mode ###
#########################
res = reduce(mergewith(+), deepcopy(vector))

##########################
### In dictionary mode ###
##########################
# res = reduce(mergewith(append!), deepcopy(vector))

Dict{Symbol, Hist1D} with 3 entries:
  :SFinZ__NN__NOMINAL => Hist1D{Float64}, edges=0.0:0.01:1.0, integral=444.3999…
  :DF__NN__NOMINAL    => Hist1D{Float64}, edges=0.0:0.01:1.0, integral=5.107544…
  :SFnoZ__NN__NOMINAL => Hist1D{Float64}, edges=0.0:0.01:1.0, integral=101.1958…

In [15]:
#########################
### In histogram mode ###
#########################
N     = nbins(res[:DF__NN__NOMINAL])
hists = rebin.([res[:SFinZ__NN__NOMINAL], res[:SFnoZ__NN__NOMINAL], res[:DF__NN__NOMINAL]], N);
@printf("Nominal yield for this ZZ file: \n")
@printf("- in SF-inZ:  %0.2f ± %0.2f\n", integral(hists[1]), only(binerrors(hists[1])))
@printf("- in SF-noZ:  %0.2f ± %0.2f\n", integral(hists[2]), only(binerrors(hists[2])))
@printf("- in DF:      %0.2f ± %0.2f\n", integral(hists[3]), only(binerrors(hists[3])))
@printf("Total:        %0.2f ± %0.2f\n", sum(map(integral,hists)), sqrt(sum(map(only∘binerrors, hists).^2)) )

##########################
### In dictionary mode ###
##########################
# @printf("Nominal yield for this ZZ file: \n")
# @printf("- in SF-inZ:  %0.2f ± %0.2f\n", sum(res[:wgt][res[:SR] .== 0]), sqrt(sum(res[:wgt][res[:SR] .== 0].^2)))
# @printf("- in SF-noZ:  %0.2f ± %0.2f\n", sum(res[:wgt][res[:SR] .== 1]), sqrt(sum(res[:wgt][res[:SR] .== 1].^2)))
# @printf("- in DF:      %0.2f ± %0.2f\n", sum(res[:wgt][res[:SR] .== 2]), sqrt(sum(res[:wgt][res[:SR] .== 2].^2)))
# @printf("Total:        %0.2f ± %0.2f\n", sum(res[:wgt]), sqrt(sum(res[:wgt].^2)))

Nominal yield for this ZZ file: 
- in SF-inZ:  444.40 ± 2.25
- in SF-noZ:  101.20 ± 1.01
- in DF:      5.11 ± 0.26
Total:        550.70 ± 2.48


- The rationale behind having an `AnalysisTask` object, is that it can be parallelized. Also this example used the nominal sample, but it can also handle systematic trees. 
- This example showed how to produce dictionaries. But by changing an argument in `AnalysisTask` we can make it produce directly simple histograms instead. Dictionaries are necessary for machine learning, while for TRExFitter we just need histograms.
- This looks a bit convoluted because I am analyzing a single file. But we have several functions in the repository that allow us to run the full analysis on all samples (with as many systematics as you want) with just a few lines of code, as shown in the next section.

# End